Since we saw good results concerning the prediction of atomization energy, we will now tried to predict the atomization energy of the polyols

In [1]:
# Train a GPR model with RDkit featurization
import deepchem as dc
import pandas as pd
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,RationalQuadratic, Matern

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
#read the descriptors that will be used to describe the polymers
file = open('../TrainingTheNN/feat_selection/feats.txt', 'r')
content = file.read()
descriptorlist = content.strip('\n').split()

In [3]:
#load the dataset (polymer smiles and their energy of atomization)

print("loading the data...")
loader1 = dc.data.CSVLoader(["Eat"], feature_field="smiles", featurizer=dc.feat.RDKitDescriptors(descriptorlist))
Data1 = loader1.create_dataset('../csv_files/Polymers_Eat.csv')

loading the data...


In [4]:
#some RDKit descriptors return nan, make these 0
X1 = np.nan_to_num(Data1.X, copy=True, nan=0.0)
#add data to dataset
Dataset = dc.data.DiskDataset.from_numpy(X=X1, y=Data1.y, w=Data1.w, ids=Data1.ids, tasks = ["Eat"])

In [5]:
#split the dataset using the random splitter
splitter = dc.splits.RandomSplitter()
train_dataset, test_dataset = splitter.train_test_split(Dataset)

# create the GPR model & fit the model
kernel = 1 * RationalQuadratic()
model = dc.models.SklearnModel(GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20))

print("fitting model...")
model.fit(train_dataset)
print("model is fitted")

#predict the test set
predicted = model.predict(test_dataset)

#calculate r2 score
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('Training set r2_score:', model.evaluate(train_dataset, metric))
test_score= model.evaluate(test_dataset, metric)
print('Test set r2_score:',round(list(test_score.values())[0],3))

#calculate RMSE score
from sklearn.metrics import root_mean_squared_error
RMSE_score = root_mean_squared_error(test_dataset.y,predicted)
print('tets set RMSE_score:',round(RMSE_score,3))

fitting model...


c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


model is fitted
Training set r2_score: {'pearson_r2_score': 0.9999999999999973}
Test set r2_score: 0.964
tets set RMSE_score: 0.066


Above the model was trained and checked, now its time to predict unknown polymers like our glycols!

In [6]:
#we will predict polyglycols with a mass from 238g/mol -987g/mol
SmileList = ["[*]CCO[*]",
"C(COCCOCCOCCOCCOCCOCCOCCO)O",
"C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O",
"C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O",
"C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O",
"C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O",
"C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O",
"C[C@@H](OC[C@@]1(c2ccccc2)CC[C@]2(CCC(=O)N2)CN1)c1cc(C(F)(F)F)cc(C(F)(F)F)c1",
"CCC(Cc1c(I)cc(I)c(N)c1I)C(=O)O",
"CN(C(=O)CC(=O)N(C)c1c(I)c(C(=O)NC(CO)C(O)CO)c(I)c(C(=O)NC(CO)C(O)CO)c1I)c1c(I)c(C(=O)NC(CO)C(O)CO)c(I)c(C(=O)NC(CO)C(O)CO)c1I",
"CC[C@H](C)C(=O)O[C@H]1C[C@H](O)C=C2C=C[C@H](C)[C@H](CC[C@@H](O)C[C@@H](O)CC(=O)O)[C@H]21",
"CC(=O)c1ccc(S(=O)(=O)NC(=O)NC2CCCCC2)cc1",
"CC(C)c1cccc(C(C)C)c1O"
]


In [7]:
#featurize these polymers
featurizer = dc.feat.RDKitDescriptors(descriptorlist)
features = featurizer.featurize(SmileList)
X_topredict = np.nan_to_num(features, copy=True, nan=0.0)
#add it to a deepchem datastructure
PredicDataset = dc.data.DiskDataset.from_numpy(X=X_topredict, ids=SmileList, tasks = ["Eat"])

In [8]:
predictions = model.predict(PredicDataset)
for idx, smiles in enumerate(SmileList):
    print(smiles,"atomization energy:",predictions[idx])

[*]CCO[*] atomization energy: -5.34921144056716
C(COCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.324097814853303
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.3071331044484396
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.355355315114139
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.39991001454473
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.421975081873825
C(COCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO)O atomization energy: -5.425174361851532
C[C@@H](OC[C@@]1(c2ccccc2)CC[C@]2(CCC(=O)N2)CN1)c1cc(C(F)(F)F)cc(C(F)(F)F)c1 atomization energy: -5.912036958819954
CCC(Cc1c(I)cc(I)c(N)c1I)C(=O)O atomization energy: -5.323301372322021
CN(C(=O)CC(=O)N(C)c1c(I)c(C(=O)NC(CO)C(O)CO)c(I)c(C(=O)NC(CO)C(O)CO)c1I)c1c(I)c(C(=O)NC(CO)C(O)CO)c(I)c(C(=O)NC(CO)C(O)CO)c1I atomization energy: -5.418012212991016
CC[C@H](C)C(=O)O[C@H]1C[C@H](O)C=C2C=C[C@H](C)[C@H](CC[C@@H](O)C[C@